<a href="https://colab.research.google.com/github/tomek-l/fire-detect-nn/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from model import Model, load_dataset, accuracy
import numpy as np

BACKBONES = ['resnet18','resnet34','resnet50','resnet101', 'densenet121', 'mobilenet']
BACKBONES = ['resnet50'] # override with just one backbone

# dunnings dataset: /home/013855803/fire-dataset-dunnings/images-224x224/train
train, valid = load_dataset()
print(f'loaded {len(train)} training batches and {len(valid)} validation batches')

loaded 114 training batches and 13 validation batches


In [2]:
train, valid = list(train), list(valid) # completely memory extravagant but I have 256GB of RAM to use :)

In [3]:
import torch
device = torch.device("cuda:0")
validate = True
for b in BACKBONES:

    import torch.optim as optim

    m = Model(backbone=b)
    m = m.to(device)

    criterion = torch.nn.BCELoss()
    optimizer = optim.Adam(m.parameters(), lr=1e-5, weight_decay=1e-5)

    for epoch in range(6):  # epochs

        running_loss = []
        running_acc = []

        # epoch training
        for i, data in enumerate(train):
            # get the inputs; data is a list of [inputs, labels]
            inputs = data[0].to(device)
            labels = data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = m(inputs)
            loss = criterion(outputs[:,0], labels.type_as(outputs[:,0]))
            loss.backward()
            optimizer.step()

            acc = accuracy(outputs, labels)
            # print statistics
            
            running_loss.append(loss.item())
            running_acc.append(acc)

            if i%20 == 19: print(f'epoch: {epoch+1}, batch: {i}, loss: {np.mean(running_loss)}, accuracy: {np.mean(running_acc)}')
        
        if validate:
            valid_acc = []
            # epoch validation
            for i, data in enumerate(valid):
                # get the inputs; data is a list of [inputs, labels]
                inputs = data[0].to(device)
                labels = data[1].to(device)

                # could pehaps do:
                # for param in m.parameters():
                #     param.requires_grad = False

                outputs = m(inputs)
                valid_acc.append(accuracy(outputs, labels))
            va = round(np.mean(valid_acc), 4)
            print(f'validation accuracy {va}')
        else:
            va = '-1'
        fname =  f'weights/{b}-epoch-{epoch}-acc={va}.pt'
        print(f'Saved {fname}')
        torch.save(m, fname)
        

    print(f'Finished Training: {b}')

epoch: 1, batch: 19, loss: 0.6273656368255616, accuracy: 0.66875
epoch: 1, batch: 39, loss: 0.5548934042453766, accuracy: 0.7765625
epoch: 1, batch: 59, loss: 0.4898707032203674, accuracy: 0.821875
epoch: 1, batch: 79, loss: 0.4304869767278433, accuracy: 0.85703125
epoch: 1, batch: 99, loss: 0.3787643563747406, accuracy: 0.88125
validation accuracy 0.9615
Saved weights/resnet50-epoch-0-acc=0.9615.pt
epoch: 2, batch: 19, loss: 0.11945381164550781, accuracy: 0.971875
epoch: 2, batch: 39, loss: 0.13745041200891137, accuracy: 0.965625
epoch: 2, batch: 59, loss: 0.12884291981657345, accuracy: 0.9666666666666667
epoch: 2, batch: 79, loss: 0.11534449709579349, accuracy: 0.9734375
epoch: 2, batch: 99, loss: 0.10198679959401488, accuracy: 0.978125
validation accuracy 0.9712
Saved weights/resnet50-epoch-1-acc=0.9712.pt
epoch: 3, batch: 19, loss: 0.03770665731281042, accuracy: 0.996875
epoch: 3, batch: 39, loss: 0.04911119150929153, accuracy: 0.990625
epoch: 3, batch: 59, loss: 0.0464549173756192